In [1]:
!pip install "/kaggle/input/autocorrect/autocorrect-2.6.1.tar"
!pip install "/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

Processing /kaggle/input/autocorrect/autocorrect-2.6.1.tar
  Preparing metadata (setup.py) ... - \ done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622383 sha256=a3e36065b9886c7aef47b567732100f02bb392934492d4c7b16209bcf12614e1
  Stored in directory: /root/.cache/pip/wheels/db/69/42/0fb0421d2fe70d195a04665edc760cfe5fd341d7bb8d8e0aaa
Successfully built autocorrect
Processing /kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl


In [2]:
from typing import List
import numpy as np
import pandas as pd
import warnings
import logging
import os
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold, GroupKFold
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
import spacy
import re
from autocorrect import Speller
from spellchecker import SpellChecker
import lightgbm as lgb

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
disable_progress_bar()
tqdm.pandas()
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(seed=3407)
DATA_DIR = "/kaggle/input/commonlit-evaluate-student-summaries/"
prompts_train = pd.read_csv(DATA_DIR + "prompts_train.csv")
prompts_test = pd.read_csv(DATA_DIR + "prompts_test.csv")
summaries_train = pd.read_csv(DATA_DIR + "summaries_train.csv")
summaries_test = pd.read_csv(DATA_DIR + "summaries_test.csv")
sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
class CFG:
    model_name_1 = "debertav3base"
    model_name_2 = "debertav3large"
    model_name_3 = "robertalarge"
    model_name_4 = "debertav3huge"#使用自己开源的huge版本
    model_name_5 = "debertav3base_single"## 使用目前开源的最高分的baseline
    
    learning_rate=0.000015  #0.000015
    weight_decay=0.03        #0.02
    hidden_dropout_prob=0.1
    attention_probs_dropout_prob=0.1
    num_train_epochs=5
    n_splits=4
    batch_size=12
    random_seed=42
    save_steps=100
    max_length=512

## Dataload

In [4]:
class Preprocessor:
    def __init__(self, 
                model_name: str,
                ) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(f"/kaggle/input/{model_name}")
        self.twd = TreebankWordDetokenizer()
        self.STOP_WORDS = set(stopwords.words('english'))
        
        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.speller = Speller(lang='en')
        self.spellchecker = SpellChecker() 
        
    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """        
        def check_is_stop_word(word):
            return word in self.STOP_WORDS
        
        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))
            
    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int) -> int:
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)
        return len(common_ngrams)
    
    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)
        
        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))
        
        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}

    
    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):
        
        wordlist=text.split()
        amount_miss = len(list(self.spellchecker.unknown(wordlist)))

        return amount_miss
    
    def add_spelling_dictionary(self, tokens: List[str]) -> List[str]:
        """dictionary update for pyspell checker and autocorrect"""
        self.spellchecker.word_frequency.load_words(tokens)
        self.speller.nlp_data.update({token:1000 for token in tokens})
    
    def run(self, 
            prompts: pd.DataFrame,
            summaries:pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:
        
        # before merge preprocess
        prompts["prompt_length"] = prompts["prompt_text"].apply(
            lambda x: len(word_tokenize(x))
        )
        prompts["prompt_tokens"] = prompts["prompt_text"].apply(
            lambda x: word_tokenize(x)
        )

        summaries["summary_length"] = summaries["text"].apply(
            lambda x: len(word_tokenize(x))
        )
        summaries["summary_tokens"] = summaries["text"].apply(
            lambda x: word_tokenize(x)
        )
        
        # Add prompt tokens into spelling checker dictionary
        prompts["prompt_tokens"].apply(
            lambda x: self.add_spelling_dictionary(x)
        )
        
#         from IPython.core.debugger import Pdb; Pdb().set_trace()
        # fix misspelling
        summaries["fixed_summary_text"] = summaries["text"].progress_apply(
            lambda x: self.speller(x)
        )
        
        # count misspelling
        summaries["splling_err_num"] = summaries["text"].progress_apply(self.spelling)
        
        # merge prompts and summaries
        input_df = summaries.merge(prompts, how="left", on="prompt_id")

        # after merge preprocess
        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']
        
        input_df['word_overlap_count'] = input_df.progress_apply(self.word_overlap_count, axis=1)
        input_df['bigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence,args=(2,), axis=1 
        )
        input_df['bigram_overlap_ratio'] = input_df['bigram_overlap_count'] / (input_df['summary_length'] - 1)
        
        input_df['trigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )
        input_df['trigram_overlap_ratio'] = input_df['trigram_overlap_count'] / (input_df['summary_length'] - 2)
        
        input_df['quotes_count'] = input_df.progress_apply(self.quotes_count, axis=1)
        
        return input_df.drop(columns=["summary_tokens", "prompt_tokens"])
    
preprocessor = Preprocessor(model_name=CFG.model_name_2)

In [5]:
# train = preprocessor.run(prompts_train, summaries_train, mode="train")
train = pd.read_csv('/kaggle/input/pre-process/train.csv')
test = preprocessor.run(prompts_test, summaries_test, mode="test")


100%|██████████| 4/4 [00:00<00:00, 2369.33it/s]


In [6]:
gkf = GroupKFold(n_splits=CFG.n_splits)

for i, (_, val_index) in enumerate(gkf.split(train, groups=train["prompt_id"])):
    train.loc[val_index, "fold"] = i

## Model Function Definition

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    preds, labels = eval_pred
    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)
    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }
def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)
    
    return (content_score + wording_score)/2

## Deberta Regressor

In [8]:
class ContentScoreRegressor:
    def __init__(self, 
                model_name: str,
                model_dir: str,
                target: str,
                hidden_dropout_prob: float,
                attention_probs_dropout_prob: float,
                max_length: int,
                ):
        self.inputs = ["prompt_text", "prompt_title", "prompt_question", "fixed_summary_text"]
        self.input_col = "input"
        
        self.text_cols = [self.input_col] 
        self.target = target
        self.target_cols = [target]

        self.model_name = model_name
        self.model_dir = model_dir
        self.max_length = max_length
        if model_name=="debertav3base":
            self.tokenizer = AutoTokenizer.from_pretrained(f"/kaggle/input/debertav3base")
            self.model_config = AutoConfig.from_pretrained(f"/kaggle/input/debertav3base")
        if model_name=="debertav3large":
            self.tokenizer = AutoTokenizer.from_pretrained(f"/kaggle/input/debertav3large")
            self.model_config = AutoConfig.from_pretrained(f"/kaggle/input/debertav3large")
        if model_name=="robertalarge":
            self.tokenizer = AutoTokenizer.from_pretrained(f"/kaggle/input/d/julian3833/robertalarge")
            self.model_config = AutoConfig.from_pretrained(f"/kaggle/input/d/julian3833/robertalarge")# /kaggle/input/robertalarge
        if model_name=="debertav3huge":
            self.tokenizer = AutoTokenizer.from_pretrained(f"/kaggle/input/debertav3large")
            self.model_config = AutoConfig.from_pretrained(f"/kaggle/input/debertav3large")
        if model_name=="debertav3base_single":
            self.tokenizer = AutoTokenizer.from_pretrained(f"/kaggle/input/debertav3base")
            self.model_config = AutoConfig.from_pretrained(f"/kaggle/input/debertav3base")
        
        self.model_config.update({
            "hidden_dropout_prob": hidden_dropout_prob,
            "attention_probs_dropout_prob": attention_probs_dropout_prob,
            "num_labels": 1,
            "problem_type": "regression",
        })
        
        seed_everything(seed=3407)

        self.data_collator = DataCollatorWithPadding(
            tokenizer=self.tokenizer
        )


    def tokenize_function(self, examples: pd.DataFrame):
        labels = [examples[self.target]]
        tokenized = self.tokenizer(examples[self.input_col],
                         padding=False,
                         truncation=True,
                         max_length=self.max_length)
        return {
            **tokenized,
            "labels": labels,
        }
    
    def tokenize_function_test(self, examples: pd.DataFrame):
        tokenized = self.tokenizer(examples[self.input_col],
                         padding=False,
                         truncation=True,
                         max_length=self.max_length)
        return tokenized

        
    def predict(self, 
                test_df: pd.DataFrame,
                fold: int,
               ):
        """predict content score"""
        
        sep = self.tokenizer.sep_token
        in_text = (
                    test_df["prompt_title"] + sep 
                    + test_df["prompt_question"] + sep 
                    + test_df["fixed_summary_text"]
                  )
        test_df[self.input_col] = in_text

        test_ = test_df[[self.input_col]]
    
        test_dataset = Dataset.from_pandas(test_, preserve_index=False) 
        test_tokenized_dataset = test_dataset.map(self.tokenize_function_test, batched=False)

        model_content = AutoModelForSequenceClassification.from_pretrained(f"{self.model_dir}")
        model_content.eval()
        
        # e.g. "bert/fold_0/"
        model_fold_dir = os.path.join(self.model_dir, str(fold)) 

        test_args = TrainingArguments(
            output_dir=model_fold_dir,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = 8,   
            dataloader_drop_last = False,
        )

        # init trainer
        infer_content = Trainer(
                      model = model_content, 
                      tokenizer=self.tokenizer,
                      data_collator=self.data_collator,
                      args = test_args)

        preds = infer_content.predict(test_tokenized_dataset)[0]

        return preds

In [9]:
def validate(
    train_df: pd.DataFrame,
    target:str,
    save_each_model: bool,
    model_name: str,
    hidden_dropout_prob: float,
    attention_probs_dropout_prob: float,
    max_length : int
    ) -> pd.DataFrame:
    """predict oof data"""
    for fold in range(CFG.n_splits):
        print(f"fold {fold}:")
        
        valid_data = train_df[train_df["fold"] == fold]
        
        if save_each_model == True:
            if model_name=="debertav3large":
                model_dir =  f"/kaggle/input/debertav3large-{target}-fold-{fold}/{target}/"
            if model_name=="debertav3base" and target=="content" :
                model_dir =  f"/kaggle/input/debertav3base-{target}-trainonly/{target}/{model_name}/fold_{fold}/"
            if model_name=="debertav3base" and target=="wording" :
                model_dir =  f"/kaggle/input/debertav3base-{target}-trainonly/{model_name}/fold_{fold}/"
            if model_name=="robertalarge":
                model_dir = f"/kaggle/input/robertalarge-{target}-{fold}/fold_{fold}"
                
            if model_name == 'debertav3huge':
                model_dir = f"/kaggle/input/debertav3large-huge/{target}/{target}/debertav3large/fold_{fold}"
        #最后使用的开源的最高分的是用的base没有分target版本
        if save_each_model == False:
            model_dir = f"/kaggle/input/commonlistdebertatuned/debertav3base/fold_{fold}"
 
        csr = ContentScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir,
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )
        
        pred = csr.predict(
            test_df=valid_data, 
            fold=fold
        )
        
        train_df.loc[valid_data.index, f"{target}_{model_name}"] = pred

    return train_df
def predict(
    test_df: pd.DataFrame,
    target:str,
    save_each_model: bool,
    model_name: str,
    hidden_dropout_prob: float,
    attention_probs_dropout_prob: float,
    max_length : int
    ):
    """predict using mean folds"""
    #模型名称是model_name
    for fold in range(CFG.n_splits):
        print(f"fold {fold}:")
        if save_each_model == True:
            if model_name=="debertav3large":
                model_dir =  f"/kaggle/input/debertav3large-{target}-fold-{fold}/{target}/"
            if model_name=="debertav3base" and target=="content" :
                model_dir =  f"/kaggle/input/debertav3base-{target}-trainonly/{target}/{model_name}/fold_{fold}/"
            if model_name=="debertav3base" and target=="wording" :
                model_dir =  f"/kaggle/input/debertav3base-{target}-trainonly/{model_name}/fold_{fold}/"
            if model_name=="robertalarge":
                model_dir = f"/kaggle/input/robertalarge-{target}-{fold}/fold_{fold}"
                
                
            if model_name == 'debertav3huge':
                model_dir = f"/kaggle/input/debertav3large-huge/{target}/{target}/debertav3large/fold_{fold}"
        #最后使用的开源的最高分的是用的base没有分target版本
        if save_each_model == False:
            model_dir = f"/kaggle/input/commonlistdebertatuned/debertav3base/fold_{fold}"
    
                
        csr = ContentScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir, 
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )
        
        pred = csr.predict(
            test_df=test_df, 
            fold=fold
        )
        
        test_df[f"{target}_{model_name}_{fold}"] = pred
    
    test_df[f"{target}_{model_name}"] = test_df[[f"{target}_{model_name}_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)

    return test_df

# 模型推断部分 

In [10]:
import time

In [11]:
start_time = time.time()
for target in ["content","wording"]:
###############################------model_name_1------#####################################################
    print('*'*30)
   
    print(f'现在是{CFG.model_name_1}_{target}的validate')
    train = validate(
        train,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name_1,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

    rmse = mean_squared_error(train[target], train[f"{target}_{CFG.model_name_1}"], squared=False)
    print(f"cv {target} rmse: {rmse}")
    print(f'现在是{CFG.model_name_1}_{target}的test')
    
    test = predict(
        test,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name_1,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )
###################################------model_name_2------#################################################
    print('*'*30)

    #model2
    print(f'现在是{CFG.model_name_2}_{target}的validate')
    
    train = validate(
        train,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name_2,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

    rmse = mean_squared_error(train[target], train[f"{target}_{CFG.model_name_2}"], squared=False)
    print(f"cv {target} rmse: {rmse}")
    print(f'现在是{CFG.model_name_2}_{target}的test')
    
    test = predict(
        test,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name_2,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )
#######################################------model_name_3------#############################################
    print('*'*30)
    
    #model3
    print(f'现在是{CFG.model_name_3}_{target}的validate')
    
    train = validate(
        train,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name_3,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

    rmse = mean_squared_error(train[target], train[f"{target}_{CFG.model_name_3}"], squared=False)
    print(f"cv {target} rmse: {rmse}")
    print(f'现在是{CFG.model_name_3}_{target}的test')
    
    test = predict(
        test,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name_3,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )
#######################################------model_name_4------#############################################
    print('*'*30)

    print(f'现在是{CFG.model_name_4}_{target}的validate')
    train = validate(
        train,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name_4,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

    rmse = mean_squared_error(train[target], train[f"{target}_{CFG.model_name_4}"], squared=False)
    print(f"cv {target} rmse: {rmse}")
    print(f'现在是{CFG.model_name_4}_{target}的test')
    
    test = predict(
        test,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name_4,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

################################------model_name_5------####################################################

    print('*'*30)

    print(f'现在是{CFG.model_name_5}_{target}的validate')
    train = validate(
        train,
        target=target,
        save_each_model=False,
        model_name=CFG.model_name_5,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

    rmse = mean_squared_error(train[target], train[f"{target}_{CFG.model_name_5}"], squared=False)
    print(f"cv {target} rmse: {rmse}")
    print(f'现在是{CFG.model_name_5}_{target}的test')
    
    test = predict(
        test,
        target=target,
        save_each_model=False,
        model_name=CFG.model_name_5,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )
    
over_tiem =time.time()
print("耗时",start_time-over_tiem)

******************************
现在是debertav3base_content的validate
fold 0:


fold 1:


fold 2:


fold 3:


cv content rmse: 0.48438723597753275
现在是debertav3base_content的test
fold 0:


...

cv wording rmse: 0.6308100650624396
现在是debertav3base_single_wording的test
fold 0:


fold 1:


fold 2:


fold 3:


耗时 -2673.059722185135


In [12]:
print(test.columns)

Index(['student_id', 'prompt_id', 'text', 'summary_length',
       'fixed_summary_text', 'splling_err_num', 'prompt_question',
       'prompt_title', 'prompt_text', 'prompt_length', 'length_ratio',
       'word_overlap_count', 'bigram_overlap_count', 'bigram_overlap_ratio',
       'trigram_overlap_count', 'trigram_overlap_ratio', 'quotes_count',
       'input', 'content_debertav3base_0', 'content_debertav3base_1',
       'content_debertav3base_2', 'content_debertav3base_3',
       'content_debertav3base', 'content_debertav3large_0',
       'content_debertav3large_1', 'content_debertav3large_2',
       'content_debertav3large_3', 'content_debertav3large',
       'content_robertalarge_0', 'content_robertalarge_1',
       'content_robertalarge_2', 'content_robertalarge_3',
       'content_robertalarge', 'content_debertav3huge_0',
       'content_debertav3huge_1', 'content_debertav3huge_2',
       'content_debertav3huge_3', 'content_debertav3huge',
       'content_debertav3base_single_0', 

In [13]:
print(train.columns)

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'summary_length', 'fixed_summary_text', 'splling_err_num',
       'prompt_question', 'prompt_title', 'prompt_text', 'prompt_length',
       'length_ratio', 'word_overlap_count', 'bigram_overlap_count',
       'bigram_overlap_ratio', 'trigram_overlap_count',
       'trigram_overlap_ratio', 'quotes_count', 'fold',
       'content_debertav3base', 'content_debertav3large',
       'content_robertalarge', 'content_debertav3huge',
       'content_debertav3base_single', 'wording_debertav3base',
       'wording_debertav3large', 'wording_robertalarge',
       'wording_debertav3huge', 'wording_debertav3base_single'],
      dtype='object')


# 在此处进行数据分析

In [14]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text", "fixed_summary_text",
                "prompt_question", "prompt_title", 
                "prompt_text"
               ] + targets


In [15]:
train[train["fold"] != 1].drop(columns=drop_columns).columns

Index(['summary_length', 'splling_err_num', 'prompt_length', 'length_ratio',
       'word_overlap_count', 'bigram_overlap_count', 'bigram_overlap_ratio',
       'trigram_overlap_count', 'trigram_overlap_ratio', 'quotes_count',
       'content_debertav3base', 'content_debertav3large',
       'content_robertalarge', 'content_debertav3huge',
       'content_debertav3base_single', 'wording_debertav3base',
       'wording_debertav3large', 'wording_robertalarge',
       'wording_debertav3huge', 'wording_debertav3base_single'],
      dtype='object')

In [16]:
model_dict = {}

for target in targets:
    models = []
    
    for fold in range(CFG.n_splits):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
            'boosting_type': 'gbdt',
            'random_state': 42,
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.048,
            'max_depth': 4,  #3
            'lambda_l1': 0.0,
            'lambda_l2': 0.011,
            'verbose':-1
        }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                               lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)
    
    model_dict[target] = models

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[54]	train's rmse: 0.400476
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.455226
Early stopping, best iteration is:
[108]	train's rmse: 0.454782
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.440447
Early stopping, best iteration is:
[87]	train's rmse: 0.439111
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.481754
[200]	train's rmse: 0.47622
Early stopping, best iteration is:
[193]	train's rmse: 0.47586
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.503573
Early stopping, best iteration is:
[105]	train's rmse: 0.503414
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.683421
Early stopping, best iteration is:
[75]	train's rmse: 0.680609
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 

In [17]:
# cv
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []
    
    for fold, model in enumerate(models):
        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)
        
    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.43887371245840207
wording_rmse : 0.5748753036305583
mcrmse : 0.5068745080444802


## Predict

In [18]:
drop_columns = [
    "student_id", "prompt_id", "text", "fixed_summary_text",
    "prompt_question", "prompt_title", 
    "prompt_text",
    "input"
] + [
    f"content_debertav3base_{i}" for i in range(CFG.n_splits)
] + [
    f"content_debertav3large_{i}" for i in range(CFG.n_splits)
] + [
    f"wording_debertav3base_{i}" for i in range(CFG.n_splits)
] + [
    f"wording_debertav3large_{i}" for i in range(CFG.n_splits)
] + [
    f"content_robertalarge_{i}" for i in range(CFG.n_splits)
] + [
    f"wording_robertalarge_{i}" for i in range(CFG.n_splits)
] + [
    f"content_debertav3huge_{i}" for i in range(CFG.n_splits)
] + [
    f"wording_debertav3huge_{i}" for i in range(CFG.n_splits)
] + [
    f"content_debertav3base_single_{i}" for i in range(CFG.n_splits)
] + [
    f"wording_debertav3base_single_{i}" for i in range(CFG.n_splits)
]


In [19]:
test.drop(columns=drop_columns).columns

Index(['summary_length', 'splling_err_num', 'prompt_length', 'length_ratio',
       'word_overlap_count', 'bigram_overlap_count', 'bigram_overlap_ratio',
       'trigram_overlap_count', 'trigram_overlap_ratio', 'quotes_count',
       'content_debertav3base', 'content_debertav3large',
       'content_robertalarge', 'content_debertav3huge',
       'content_debertav3base_single', 'wording_debertav3base',
       'wording_debertav3large', 'wording_robertalarge',
       'wording_debertav3huge', 'wording_debertav3base_single'],
      dtype='object')

# 使用LGB进行预测

In [20]:

pred_dict = {}  # 创建一个空字典，用于存储预测结果
for target in targets:  
    models = model_dict[target]  # 获取当前目标的模型列表
    preds = []  # 创建一个空列表，用于存储预测结果
    for fold, model in enumerate(models):  # 遍历当前目标的每个折中的模型
        X_eval_cv = test.drop(columns=drop_columns)  # 获取测试数据，并删除不需要的列
        pred = model.predict(X_eval_cv)  # 对测试数据进行预测#LGB
        preds.append(pred)  # 将预测结果添加到列表中
    pred_dict[target] = preds  # 将当前目标的预测结果列表存储到预测字典中


In [21]:
pred_dict

{'content': [array([-1.22840834, -1.22840834, -1.22840834, -1.22840834]),
  array([-1.13469705, -1.13469705, -1.13469705, -1.13469705]),
  array([-1.58071948, -1.58071948, -1.58071948, -1.58071948]),
  array([-1.62929385, -1.62929385, -1.65796862, -1.65796862])],
 'wording': [array([-1.35832507, -1.35832507, -1.35832507, -1.34417148]),
  array([-1.19091336, -1.19091336, -1.19091336, -1.19091336]),
  array([-1.48859074, -1.48859074, -1.48859074, -1.47599594]),
  array([-1.44453214, -1.44453214, -1.44927576, -1.43232973])]}

In [22]:
for target in targets:  # 遍历目标列表中的每一个目标
    preds = pred_dict[target]  # 获取当前目标的预测结果列表#LGB的预测结果
    
    for i, pred in enumerate(preds):  # 遍历当前目标的每个折中的预测结果
        test[f"{target}_pred_{i}"] = pred  # 将预测结果添加到测试数据中，创建新的列名
    test[f'{target}'] = test[[f"{target}_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)        

In [23]:
test

,student_id,prompt_id,text,summary_length,fixed_summary_text,splling_err_num,prompt_question,prompt_title,prompt_text,prompt_length,...,content_pred_0,content_pred_1,content_pred_2,content_pred_3,content,wording_pred_0,wording_pred_1,wording_pred_2,wording_pred_3,wording
0,000000ffffff,abc123,Example text 1,3,Example text 1,0,Summarize...,Example Title 1,Heading\nText...,3,...,-1.228408,-1.134697,-1.580719,-1.629294,-1.393280,-1.358325,-1.190913,-1.488591,-1.444532,-1.370590
1,111111eeeeee,def789,Example text 2,3,Example text 2,0,Summarize...,Example Title 2,Heading\nText...,3,...,-1.228408,-1.134697,-1.580719,-1.629294,-1.393280,-1.358325,-1.190913,-1.488591,-1.444532,-1.370590
2,222222cccccc,abc123,Example text 3,3,Example text 3,0,Summarize...,Example Title 1,Heading\nText...,3,...,-1.228408,-1.134697,-1.580719,-1.657969,-1.400448,-1.358325,-1.190913,-1.488591,-1.449276,-1.371776
3,333333dddddd,def789,Example text 4,3,Example text 4,0,Summarize...,Example Title 2,Heading\nText...,3,...,-1.228408,-1.134697,-1.580719,-1.657969,-1.400448,-1.344171,-1.190913,-1.475996,-1.432330,-1.360853


## Create Submission file

In [24]:
test[["student_id", "content", "wording"]].to_csv("submission.csv", index=False)